[link](http://hlm.lge.com/issue/browse/HEDATAPLFM-896?attachmentSortBy=dateTime&attachmentOrder=asc)

글로벌 데이터 추출


- 2개 조건의 로그를 각각 추출하고 (CL0)
- mac기준으로 결합하여 제공 (CL1)

In [0]:

CREATE TABLE IF NOT EXISTS adhoc.heds.896_cl0 AS
WITH TB0 AS (
  SELECT
    explode(from_json(normal_log:dvc_list, 'array<struct<name:string, type:string, spec:string, label:string, id:string>>')) AS dvc,
    *
  FROM kic_data_ods.tlamp.normal_log_webos24
  WHERE 
    date_ym BETWEEN '2024-03' AND '2025-06'
    AND context_name = 'com.webos.app.homeconnect'
    AND message_id = 'NL_HC_INPUTLIST'
    AND normal_log:dvc_list != '[]'
),
TB1 AS (
  SELECT
    date_ym,
    log_create_time,
    X_Device_Sales_Model AS tv_model,
    DEVICE_NAME AS soc,
    mac_addr,
    dvc.type,
    dvc.label,
    dvc.name,
    ROW_NUMBER() OVER (PARTITION BY mac_addr ORDER BY log_create_time DESC) AS rn
  FROM TB0
  WHERE dvc.type = 'audio'
)
SELECT
  date_ym,
  mac_addr,
  tv_model,
  soc,
  TB1.type as sb_type,
  TB1.label as sb_label,
  TB1.name as sb_name,
  log_create_time as NH_HC_INPUTLIST_last_log_dt
FROM TB1
WHERE rn = 1
;
-- LIMIT 500
;

create table if not exists adhoc.heds.896_cl0_2 as
with tb0 as (
  select 
    normal_log:changed_value as soundAlive_status,
    mac_addr,
    log_create_time,
    row_number() over (partition by mac_addr order by log_create_time desc) as rn
  from kic_data_ods.tlamp.normal_log_webos24
  where 1=1
    and date_ym BETWEEN '2024-03' AND '2025-06'
    and context_name = 'com.palm.app.settings'
    and message_id in ('NL_SETTING_CHANGE', 'INPUTLIST')
    and normal_log:menu_name ='soundAlive'
)
select 
  mac_addr, 
  log_create_time as soundAlive_last_log_dt,
  soundAlive_status as soundAlive_last_status
from tb0
where rn = 1
;

In [0]:
select 
  t1.* except (t1.mac_addr), t2.* except (t2.mac_addr)
from adhoc.heds.896_cl0 t1
left join adhoc.heds.896_cl0_2 t2 using (mac_addr)
